In [1]:
pip install scikit-surprise==1.1.3

Note: you may need to restart the kernel to use updated packages.


# Домашнє завдання модуля

In [2]:
import pandas as pd
from surprise import accuracy, Dataset, SVD, SVDpp, NMF
from surprise.model_selection import train_test_split
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [3]:
dataset = Dataset.load_builtin('ml-100k')

In [4]:
train = dataset.build_full_trainset()
algoritms = [SVD, SVDpp, NMF]
param= {'n_factors': [2,4], 'n_epochs': [5,10]}
best_algoritm = None
best_params = None
best_score = float('inf')

for algoritm in algoritms:
    grid_search = GridSearchCV(algoritm, param, measures=["rmse"], cv=5)
    grid_search.fit(dataset)
    
    print(f"Найкращий результат RMSE для {algoritm}: {grid_search.best_score['rmse']}")
    print(f"Найкращі параметри: {grid_search.best_params['rmse']}")
    
    if grid_search.best_score['rmse'] < best_score:
        best_score = grid_search.best_score['rmse']
        best_params = grid_search.best_params['rmse']
        best_algoritm = algoritm

print(f"\nНайкращий алгоритм в цілому: {best_algoritm}")
print(f"Найкращий результат RMSE: {best_score}")
print(f"Найкращі параметри: {best_params}")

Найкращий результат RMSE для <class 'surprise.prediction_algorithms.matrix_factorization.SVD'>: 0.9455764813480441
Найкращі параметри: {'n_factors': 2, 'n_epochs': 10}
Найкращий результат RMSE для <class 'surprise.prediction_algorithms.matrix_factorization.SVDpp'>: 0.9339557338119613
Найкращі параметри: {'n_factors': 4, 'n_epochs': 10}
Найкращий результат RMSE для <class 'surprise.prediction_algorithms.matrix_factorization.NMF'>: 1.7972089917348035
Найкращі параметри: {'n_factors': 4, 'n_epochs': 5}

Найкращий алгоритм в цілому: <class 'surprise.prediction_algorithms.matrix_factorization.SVDpp'>
Найкращий результат RMSE: 0.9339557338119613
Найкращі параметри: {'n_factors': 4, 'n_epochs': 10}


# Додаткове завдання з зірочкою

In [5]:
import numpy as np
from scipy import optimize
from scipy.io import loadmat

In [6]:
movie_ids = 'movie_ids.txt'
movies_mat = 'movies.mat'

In [7]:
def normalizeRatings(Y, R):
    m, n = Y.shape
    Ymean = np.zeros(m)
    Ynorm = np.zeros(Y.shape)

    for i in range(m):
        idx = R[i, :] == 1
        Ymean[i] = np.mean(Y[i, idx])
        Ynorm[i, idx] = Y[i, idx] - Ymean[i]

    return Ynorm, Ymean


def loadMovieList():
    with open(movie_ids,  encoding='ISO-8859-1') as fid:
        movies = fid.readlines()

    movieNames = []
    for movie in movies:
        parts = movie.split()
        movieNames.append(' '.join(parts[1:]).strip())
    return movieNames


def computeNumericalGradient(J, theta, e=1e-4):
    numgrad = np.zeros(theta.shape)
    perturb = np.diag(e * np.ones(theta.shape))
    for i in range(theta.size):
        loss1, _ = J(theta - perturb[:, i])
        loss2, _ = J(theta + perturb[:, i])
        numgrad[i] = (loss2 - loss1)/(2*e)
    return numgrad


def checkCostFunction(cofiCostFunc, lambda_=0.):

    X_t = np.random.rand(4, 3)
    Theta_t = np.random.rand(5, 3)

    Y = np.dot(X_t, Theta_t.T)
    Y[np.random.rand(*Y.shape) > 0.5] = 0
    R = np.zeros(Y.shape)
    R[Y != 0] = 1

    X = np.random.randn(*X_t.shape)
    Theta = np.random.randn(*Theta_t.shape)
    num_movies, num_users = Y.shape
    num_features = Theta_t.shape[1]

    params = np.concatenate([X.ravel(), Theta.ravel()])
    numgrad = computeNumericalGradient(
        lambda x: cofiCostFunc(x, Y, R, num_users, num_movies, num_features, lambda_), params)

    cost, grad = cofiCostFunc(params, Y, R, num_users,num_movies, num_features, lambda_)

    print(np.stack([numgrad, grad], axis=1))
    print('\nНаведені вище два стовпці мають бути дуже схожими.'
          '\n(Ліворуч — наш числовий градієнт, праворуч — аналітичний градієнт)')

    diff = np.linalg.norm(numgrad-grad)/np.linalg.norm(numgrad+grad)
    print('Якщо наша функція витрат реалізована правильно, тоді '
          'відносна різниця буде невеликою (менше 1e-9).')
    print('\nВідносна різниця: %g' % diff)

In [8]:
names = loadMovieList()
len(names)

1682

In [9]:
data = loadmat(movies_mat)
Y, R = data['Y'], data['R']

print('Середній рейтинг фільму 1302 (',names[1302] ,'): %f / 5' %
      np.mean(Y[180, R[1400, :]]))

Середній рейтинг фільму 1302 ( Getaway, The (1994) ): 4.904560 / 5


In [10]:
def cofiCostFunc(params, Y, R, num_users, num_movies,
                      num_features, lambda_=0.0):
    # Розгортаємо матриці U та W з params
    X = params[:num_movies*num_features].reshape(num_movies, num_features)
    Theta = params[num_movies*num_features:].reshape(num_users, num_features)

    J = 0
    X_grad = np.zeros(X.shape)
    Theta_grad = np.zeros(Theta.shape)

    J = (1 / 2) * np.sum(np.square((X.dot(Theta.T) - Y) * R)) + (lambda_ / 2) * np.sum(np.square(X)) + \
                                                                (lambda_ / 2) * np.sum(np.square(Theta))
    
    for i in range(R.shape[0]):
        
        idx = np.where(R[i, :] == 1)[0]
        Theta_temp = Theta[idx, :]
        Y_temp = Y[i, idx]
        X_grad[i, :] = np.dot(np.dot(X[i, :], Theta_temp.T) - Y_temp, Theta_temp) + lambda_ * X[i, :]
        
    for j in range(R.shape[1]):
        
        idx = np.where(R[:, j] == 1)[0]
        X_temp = X[idx, :]
        Y_temp = Y[idx, j]
        Theta_grad[j, :] = np.dot(np.dot(X_temp, Theta[j, :]) - Y_temp, X_temp) + lambda_ * Theta[j, :]
    
    grad = np.concatenate([X_grad.ravel(), Theta_grad.ravel()])
    return J, grad

In [11]:
checkCostFunction(cofiCostFunc, 1.5)

[[  5.89472311   5.89472311]
 [  1.75464126   1.75464126]
 [ -3.47121562  -3.47121562]
 [  1.21202929   1.21202929]
 [ -2.6395556   -2.6395556 ]
 [ -2.04319608  -2.04319607]
 [-10.64118877 -10.64118877]
 [  3.80093391   3.80093391]
 [  6.3372085    6.3372085 ]
 [  7.01903894   7.01903894]
 [ -5.50555454  -5.50555454]
 [ -1.80078576  -1.80078576]
 [-15.92048987 -15.92048987]
 [ -1.52711616  -1.52711616]
 [  4.47355202   4.47355202]
 [ 14.65222667  14.65222667]
 [ -3.76987769  -3.76987769]
 [ -3.72711961  -3.72711961]
 [ -9.37029888  -9.37029888]
 [  1.21023573   1.21023573]
 [  2.6713645    2.6713645 ]
 [  2.57905426   2.57905426]
 [ -0.98566219  -0.98566219]
 [  1.68884641   1.68884641]
 [  3.792902     3.792902  ]
 [  2.55849461   2.55849461]
 [ -0.57790214  -0.57790214]]

Наведені вище два стовпці мають бути дуже схожими.
(Ліворуч — наш числовий градієнт, праворуч — аналітичний градієнт)
Якщо наша функція витрат реалізована правильно, тоді відносна різниця буде невеликою (менше 1e-9)

In [12]:
import random

In [13]:
movieList = loadMovieList()

#Перш ніж навчати модель спільної фільтрації, ми спочатку
# додамо оцінки, які відповідають новому користувачеві

n_m = len(movieList)
# Ініціалізуємо оцінки
my_ratings = np.zeros(n_m)

my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)
my_ratings[random.randint(0, 1682)] = random.randint(1, 5)

In [14]:
# Додаємо наші оцінки до матриці даних
Y = np.hstack([my_ratings[:, None], Y])
R = np.hstack([(my_ratings > 0)[:, None], R])

# Нормалізація рейтингів
Ynorm, Ymean = normalizeRatings(Y, R)

# Корисні цінності
num_movies, num_users = Y.shape
num_features = 7

# Встановлюємо початкові параметри (Theta, X)
X = np.random.randn(num_movies, num_features)
Theta = np.random.randn(num_users, num_features)

initial_parameters = np.concatenate([X.ravel(), Theta.ravel()])

# Встановлюємо параметри для scipy.optimize.minimize
options = {'maxiter': 100}

# Встановлюємо регулярізацію
lambda_ = 10
res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,
                                               num_movies, num_features, lambda_),
                        initial_parameters,
                        method='TNC',
                        jac=True,
                        options=options)
theta = res.x

# Розгортаємо повернуту Theta назад в U та W
X = theta[:num_movies*num_features].reshape(num_movies, num_features)
Theta = theta[num_movies*num_features:].reshape(num_users, num_features)

/var/folders/gl/lm7vwnyj6nl9kckblky8tx_w0000gn/T/ipykernel_63533/2851689147.py:23: OptimizeWarning: Unknown solver options: maxiter
  res = optimize.minimize(lambda x: cofiCostFunc(x, Ynorm, R, num_users,


In [15]:
# Зробимо рекомендації, обчислюючи матрицю прогнозів
p = np.dot(X, Theta.T)
my_predictions = p[:, 0] + Ymean

movieList = loadMovieList()

ix = np.argsort(my_predictions)[::-1]

print('Найкращі рекомендації для вас:')
print('----------------------------')
for i in range(15):
    j = ix[i]
    print('Спрогнозований рейтингу %.1f для фільму %s' % (my_predictions[j], movieList[j]))

print('\nОригінальні рейтинги:')
print('--------------------------')
for i in range(len(my_ratings)):
    if my_ratings[i] > 0:
        print('Рейтинг %d для фільму %s' % (my_ratings[i], movieList[i]))

Найкращі рекомендації для вас:
----------------------------
Спрогнозований рейтингу 5.0 для фільму Great Day in Harlem, A (1994)
Спрогнозований рейтингу 5.0 для фільму Aiqing wansui (1994)
Спрогнозований рейтингу 5.0 для фільму Entertaining Angels: The Dorothy Day Story (1996)
Спрогнозований рейтингу 5.0 для фільму Star Kid (1997)
Спрогнозований рейтингу 5.0 для фільму Prefontaine (1997)
Спрогнозований рейтингу 5.0 для фільму Saint of Fort Washington, The (1993)
Спрогнозований рейтингу 5.0 для фільму Marlene Dietrich: Shadow and Light (1996)
Спрогнозований рейтингу 5.0 для фільму Someone Else's America (1995)
Спрогнозований рейтингу 5.0 для фільму Santa with Muscles (1996)
Спрогнозований рейтингу 5.0 для фільму They Made Me a Criminal (1939)
Спрогнозований рейтингу 4.6 для фільму Pather Panchali (1955)
Спрогнозований рейтингу 4.5 для фільму Everest (1998)
Спрогнозований рейтингу 4.5 для фільму Some Mother's Son (1996)
Спрогнозований рейтингу 4.5 для фільму Anna (1996)
Спрогнозований ре